<a href="https://colab.research.google.com/github/teacherSsamko/DL-study/blob/main/w5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [5주차] 기본과제: 외부 블로그의 정보와 함께 챗봇 구현하기

이번 실습에서는 LangChain으로 개발한 RAG를 다음 블로그의 정보와 연동하면 됩니다. 요구사항은 다음과 같습니다:

- RAG internet source를 https://spartacodingclub.kr/blog/all-in-challenge_winner 로 설정.
    - RAG에서 활용할 source로 위의 링크를 전달합니다.
    - 사이트가 달라졌기 때문에 이전 실습 코드와 다르게 load 해야 합니다. 어디를 어떻게 수정해야 할지 고민해보도록 합시다.
    - LLM은 GPT를 사용하시면 됩니다. 모델은 `gpt-4o-mini`로 설정하시면 됩니다.
- GPT에게 `“ALL-in 코딩 공모전 수상작들을 요약해줘.”`를 물은 뒤의 답변을 출력.

In [1]:
!pip install langchain-community langchain-chroma langchain-openai bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.8 MB/s eta 0:00:00
   

In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

## [MY CODE] 환경 변수 설정

### OpenAI

ChatOpenAI는 OPENAI_API_KEY를 환경변수로만 받고 있어서, 환경변수에 지정

관련 문서: [ChatOpenAI](https://python.langchain.com/docs/integrations/chat/openai/)

## LANGCHAIN

LANGCHAIN과 LANGSMITH 관련 환경변수 설정

관련 문서: [LANGSMITH](https://docs.smith.langchain.com/), [LANGCHAIN](https://smith.langchain.com/o/464e449d-6cf3-4e7a-a47f-5b96b912650d?paginationState=%7B%22pageIndex%22:0,%22pageSize%22:5%7D)

In [37]:
import os

from google.colab import userdata
import openai



# openai API KEY
openai_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

# langsmith
langsmith_key = userdata.get('LANGSMITH_KEY')
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = langsmith_key
os.environ["LANGSMITH_API_KEY"] = langsmith_key
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "sparta_w5_1"


llm = ChatOpenAI(model="gpt-4o-mini")

## [MY CODE] encoding 설정

encoding 설정이 없으면, 한글이 깨져서, utf-8으로 설정

In [22]:
web_source = "https://spartacodingclub.kr/blog/all-in-challenge_winner"

loader = WebBaseLoader(
    web_paths=(web_source,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("editedContent",)
        )
    ),
    encoding="utf-8"
)
docs = loader.load()
docs

[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로

In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=openai_key)
)

## [MY CODE] 함수는 재사용을 위해 별도로 관리

In [24]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [38]:
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 배경을 가진 학생들이 함께 성장할 수 있는 환경이 조성됩니다. 글로벌 사회에서 중요한 소통 능력을 기를 수 있는 문화의 장으로 쓰일 수 있겠죠!사용한 기술 스택-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket-BE(백엔드): React Native, TanStack Query, Axios코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도전해 보세요.💡<All-in> 코딩 공모전에서 만든 다양한 서비스를 만나보고 싶다면?다양한 서비스와 기발한 아이디어가 모인 곳에 초대합니다. 참가자들의 문제 해결방법이 궁금하시다면 지금 바로 ‘All-in 공모전’에서 만나보세요!👉🏻\xa0공모전 결과물 보러가기누구나 큰일 낼 수 있어스파르타코딩클럽글 | 신수지 팀스파르타 에디터\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [31]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전에서는 대학생들이 캠퍼스에서의 문제를 해결하기 위한 혁신적인 웹/앱 서비스를 개발했습니다. 주요 수상작으로는 언어 공부를 도와주는 'Lexi Note', 연합동아리 정보 플랫폼 'Crewing', 학교생활 관리 서비스 '학교생활 매니저', 그리고 실시간 소통을 지원하는 '에코 클래스룸' 등이 있습니다. 이 공모전은 창의적인 아이디어와 코딩 기술을 통해 문제 해결 능력을 배양할 수 있는 기회를 제공하였습니다.


In [32]:
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 대상 수상작에 대해 설명해줘"
retrieved_docs = retriever.invoke(user_msg)

prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

response = llm.invoke(user_prompt)
print(response.content)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 대상 수상작에 대해 설명해줘 \nContext: 배경을 가진 학생들이 함께 성장할 수 있는 환경이 조성됩니다. 글로벌 사회에서 중요한 소통 능력을 기를 수 있는 문화의 장으로 쓰일 수 있겠죠!사용한 기술 스택-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket-BE(백엔드): React Native, TanStack Query, Axios코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도전해 보세요.💡<All-in> 코딩 공모전에서 만든 다양한 서비스를 만나보고 싶다면?다양한 서비스와 기발한 아이디어가 모인 곳에 초대합니다. 참가자들의 문제 해결방법이 궁금하시다면 지금 바로 ‘All-in 공모전’에서 만나보세요!👉🏻\xa0공모전 결과물 보러가기누구나 큰일 낼 수 있어스파르타코딩클럽글 | 신수지 팀스파르타 에디터\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상